# Space Titanic - 우주 타이타닉 생존자 예측

이 노트북은 Space Titanic 데이터셋을 사용하여 승객이 다른 차원으로 전송되었는지 예측하는 분류 모델을 구축합니다.

## 목차
1. 라이브러리 및 데이터 로드
2. 탐색적 데이터 분석 (EDA)
3. 데이터 전처리
4. 특성 공학 (Feature Engineering)
5. 모델링
6. 예측 및 제출 파일 생성

---
## 1. 라이브러리 및 데이터 로드

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리 및 모델링
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style('whitegrid')

print("라이브러리 로드 완료!")

In [ ]:
# 데이터 로드
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

print(f"학습 데이터 크기: {train.shape}")
print(f"테스트 데이터 크기: {test.shape}")
print(f"제출 파일 크기: {submission.shape}")

---
## 2. 탐색적 데이터 분석 (EDA)

### 2.1 데이터 기본 정보 확인

In [ ]:
# 데이터 상위 5개 행 확인
train.head()

In [ ]:
# 데이터 타입 및 결측치 확인
train.info()

In [ ]:
# 수치형 변수 통계량
train.describe()

In [ ]:
# 범주형 변수 통계량
train.describe(include='object')

### 2.2 결측치 분석

In [ ]:
# 결측치 비율 확인
def show_missing(df, name):
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        '결측치 수': missing,
        '결측치 비율(%)': missing_pct.round(2)
    })
    missing_df = missing_df[missing_df['결측치 수'] > 0].sort_values('결측치 비율(%)', ascending=False)
    print(f"\n=== {name} 결측치 현황 ===")
    return missing_df

show_missing(train, '학습 데이터')

In [ ]:
# 결측치 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 학습 데이터 결측치
train_missing = train.isnull().sum()
train_missing = train_missing[train_missing > 0].sort_values(ascending=True)
axes[0].barh(train_missing.index, train_missing.values, color='coral')
axes[0].set_title('Train - 결측치 수')
axes[0].set_xlabel('결측치 수')

# 테스트 데이터 결측치
test_missing = test.isnull().sum()
test_missing = test_missing[test_missing > 0].sort_values(ascending=True)
axes[1].barh(test_missing.index, test_missing.values, color='steelblue')
axes[1].set_title('Test - 결측치 수')
axes[1].set_xlabel('결측치 수')

plt.tight_layout()
plt.show()

### 2.3 타겟 변수 분석

In [ ]:
# 타겟 변수 분포
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 카운트
train['Transported'].value_counts().plot(kind='bar', ax=axes[0], color=['salmon', 'lightgreen'])
axes[0].set_title('Transported 분포')
axes[0].set_xlabel('Transported')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['False', 'True'], rotation=0)

# 비율
train['Transported'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', 
                                         colors=['salmon', 'lightgreen'])
axes[1].set_title('Transported 비율')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

print(f"\nTransported 비율:\n{train['Transported'].value_counts(normalize=True).round(3)}")

### 2.4 범주형 변수 분석

In [ ]:
# 범주형 변수와 타겟 변수의 관계
cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, col in enumerate(cat_cols):
    # 타겟별 비율 계산
    ct = pd.crosstab(train[col], train['Transported'], normalize='index') * 100
    ct.plot(kind='bar', ax=axes[i], color=['salmon', 'lightgreen'])
    axes[i].set_title(f'{col}별 Transported 비율')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('비율 (%)')
    axes[i].legend(['Not Transported', 'Transported'])
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

**인사이트:**
- CryoSleep이 True인 승객의 Transported 비율이 훨씬 높음
- Europa 출신 승객의 Transported 비율이 높음
- VIP 여부는 큰 영향이 없어 보임

### 2.5 수치형 변수 분석

In [ ]:
# 지출 관련 변수
spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Age 분포
sns.histplot(data=train, x='Age', hue='Transported', kde=True, ax=axes[0])
axes[0].set_title('Age 분포')

# 지출 변수 분포
for i, col in enumerate(spend_cols):
    # 로그 변환하여 시각화 (0은 제외)
    temp = train[train[col] > 0][col]
    axes[i+1].hist(temp, bins=50, color='steelblue', alpha=0.7)
    axes[i+1].set_title(f'{col} 분포 (0 제외)')
    axes[i+1].set_xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# 지출 변수와 타겟 변수의 관계
fig, axes = plt.subplots(1, 5, figsize=(18, 4))

for i, col in enumerate(spend_cols):
    sns.boxplot(data=train, x='Transported', y=col, ax=axes[i])
    axes[i].set_title(f'{col} vs Transported')

plt.tight_layout()
plt.show()

**인사이트:**
- 지출이 많은 승객일수록 Transported=False인 경향이 있음
- 이는 CryoSleep 상태의 승객이 지출을 할 수 없기 때문으로 추정

### 2.6 상관관계 분석

In [ ]:
# 수치형 변수 상관관계
numeric_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# 타겟 변수 추가 (True=1, False=0)
train_corr = train[numeric_cols].copy()
train_corr['Transported'] = train['Transported'].astype(int)

plt.figure(figsize=(10, 8))
sns.heatmap(train_corr.corr(), annot=True, cmap='RdBu_r', center=0, fmt='.2f')
plt.title('상관관계 히트맵')
plt.show()

---
## 3. 데이터 전처리

In [ ]:
# 학습/테스트 데이터 합치기 (동일한 전처리 적용)
train['is_train'] = 1
test['is_train'] = 0

df = pd.concat([train, test], axis=0, ignore_index=True)
print(f"결합된 데이터 크기: {df.shape}")

### 3.1 Cabin 변수 분리

In [ ]:
# Cabin = deck/num/side 형식
# 예: B/0/P -> Deck=B, CabinNum=0, Side=P

df['Deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if pd.notna(x) else np.nan)
df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if pd.notna(x) else np.nan)
df['Side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if pd.notna(x) else np.nan)

print("Deck 고유값:", df['Deck'].unique())
print("Side 고유값:", df['Side'].unique())

### 3.2 PassengerId에서 그룹 정보 추출

In [ ]:
# PassengerId = 그룹ID_그룹내번호 형식
# 예: 0001_01 -> GroupId=0001, PersonNum=01

df['GroupId'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
df['PersonNum'] = df['PassengerId'].apply(lambda x: int(x.split('_')[1]))

# 그룹 크기 계산
group_size = df.groupby('GroupId').size()
df['GroupSize'] = df['GroupId'].map(group_size)

print(f"그룹 크기 분포:\n{df['GroupSize'].value_counts().sort_index()}")

### 3.3 결측치 처리

In [ ]:
# 결측치 처리 함수
def fill_missing(df):
    # 범주형 변수: 최빈값으로 채우기
    cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
    for col in cat_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    # 수치형 변수: 중앙값으로 채우기
    num_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum']
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())
    
    return df

df = fill_missing(df)

# 결측치 확인
print("결측치 처리 후:")
print(df[['HomePlanet', 'CryoSleep', 'Age', 'RoomService', 'Deck', 'Side']].isnull().sum())

---
## 4. 특성 공학 (Feature Engineering)

### 4.1 총 지출액 및 지출 관련 특성

In [ ]:
# 지출 관련 변수
spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# 총 지출액
df['TotalSpent'] = df[spend_cols].sum(axis=1)

# 지출 여부 (0: 지출 없음, 1: 지출 있음)
df['HasSpent'] = (df['TotalSpent'] > 0).astype(int)

# 지출 항목 수
df['SpendCount'] = (df[spend_cols] > 0).sum(axis=1)

print("총 지출액 통계:")
print(df['TotalSpent'].describe())

### 4.2 나이 그룹 생성

In [ ]:
# 나이 그룹 분류
def age_group(age):
    if age < 12:
        return 'Child'
    elif age < 18:
        return 'Teen'
    elif age < 30:
        return 'Young_Adult'
    elif age < 50:
        return 'Adult'
    else:
        return 'Senior'

df['AgeGroup'] = df['Age'].apply(age_group)

print("나이 그룹 분포:")
print(df['AgeGroup'].value_counts())

### 4.3 혼자 여행 여부

In [ ]:
# 혼자 여행하는 승객
df['IsAlone'] = (df['GroupSize'] == 1).astype(int)

print(f"혼자 여행: {df['IsAlone'].sum()}명")
print(f"그룹 여행: {(df['IsAlone'] == 0).sum()}명")

### 4.4 범주형 변수 인코딩

In [ ]:
# 범주형 변수 리스트
cat_cols_to_encode = ['HomePlanet', 'Destination', 'Deck', 'Side', 'AgeGroup']

# Boolean 변수 처리
df['CryoSleep'] = df['CryoSleep'].astype(bool).astype(int)
df['VIP'] = df['VIP'].astype(bool).astype(int)

# Label Encoding
label_encoders = {}
for col in cat_cols_to_encode:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

print("인코딩 완료!")
print(f"\nHomePlanet 인코딩: {dict(zip(label_encoders['HomePlanet'].classes_, range(len(label_encoders['HomePlanet'].classes_))))}")

### 4.5 최종 특성 선택

In [ ]:
# 모델에 사용할 특성 선택
feature_cols = [
    # 기본 특성
    'Age', 'CryoSleep', 'VIP',
    # 지출 관련
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
    'TotalSpent', 'HasSpent', 'SpendCount',
    # 그룹 관련
    'GroupSize', 'IsAlone',
    # 인코딩된 범주형
    'HomePlanet_encoded', 'Destination_encoded', 
    'Deck_encoded', 'Side_encoded', 'AgeGroup_encoded'
]

print(f"총 특성 수: {len(feature_cols)}")
print(f"특성 목록: {feature_cols}")

---
## 5. 모델링

### 5.1 데이터 분리

In [ ]:
# 학습/테스트 데이터 분리
train_df = df[df['is_train'] == 1].copy()
test_df = df[df['is_train'] == 0].copy()

# 특성과 타겟 분리
X = train_df[feature_cols]
y = train_df['Transported'].astype(int)

X_test = test_df[feature_cols]

print(f"학습 특성 크기: {X.shape}")
print(f"타겟 크기: {y.shape}")
print(f"테스트 특성 크기: {X_test.shape}")

In [ ]:
# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"검증 데이터 크기: {X_val.shape}")

### 5.2 Random Forest 모델

In [ ]:
# Random Forest 모델 학습
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# 검증 데이터 예측
rf_pred = rf_model.predict(X_val)
rf_accuracy = accuracy_score(y_val, rf_pred)

print(f"Random Forest 검증 정확도: {rf_accuracy:.4f}")

In [ ]:
# 교차 검증
rf_cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')
print(f"Random Forest 교차 검증 점수: {rf_cv_scores}")
print(f"평균 교차 검증 점수: {rf_cv_scores.mean():.4f} (+/- {rf_cv_scores.std()*2:.4f})")

### 5.3 Gradient Boosting 모델

In [ ]:
# Gradient Boosting 모델 학습
gb_model = GradientBoostingClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

gb_model.fit(X_train, y_train)

# 검증 데이터 예측
gb_pred = gb_model.predict(X_val)
gb_accuracy = accuracy_score(y_val, gb_pred)

print(f"Gradient Boosting 검증 정확도: {gb_accuracy:.4f}")

In [ ]:
# 교차 검증
gb_cv_scores = cross_val_score(gb_model, X, y, cv=5, scoring='accuracy')
print(f"Gradient Boosting 교차 검증 점수: {gb_cv_scores}")
print(f"평균 교차 검증 점수: {gb_cv_scores.mean():.4f} (+/- {gb_cv_scores.std()*2:.4f})")

### 5.4 모델 비교

In [ ]:
# 모델 비교
results = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting'],
    'Validation Accuracy': [rf_accuracy, gb_accuracy],
    'CV Mean': [rf_cv_scores.mean(), gb_cv_scores.mean()],
    'CV Std': [rf_cv_scores.std(), gb_cv_scores.std()]
})

print(results.to_string(index=False))

### 5.5 특성 중요도

In [ ]:
# 더 좋은 모델의 특성 중요도 시각화
best_model = gb_model if gb_accuracy > rf_accuracy else rf_model
best_model_name = 'Gradient Boosting' if gb_accuracy > rf_accuracy else 'Random Forest'

# 특성 중요도
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
plt.xlabel('중요도')
plt.title(f'{best_model_name} 특성 중요도')
plt.tight_layout()
plt.show()

### 5.6 혼동 행렬

In [ ]:
# 혼동 행렬 시각화
best_pred = gb_pred if gb_accuracy > rf_accuracy else rf_pred

cm = confusion_matrix(y_val, best_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Not Transported', 'Transported'],
            yticklabels=['Not Transported', 'Transported'])
plt.xlabel('예측')
plt.ylabel('실제')
plt.title(f'{best_model_name} 혼동 행렬')
plt.tight_layout()
plt.show()

print(f"\n{best_model_name} 분류 보고서:")
print(classification_report(y_val, best_pred, target_names=['Not Transported', 'Transported']))

---
## 6. 예측 및 제출 파일 생성

In [ ]:
# 전체 학습 데이터로 최종 모델 학습
final_model = GradientBoostingClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

final_model.fit(X, y)

# 테스트 데이터 예측
test_pred = final_model.predict(X_test)

print(f"테스트 예측 완료!")
print(f"예측 결과 분포:\n{pd.Series(test_pred).value_counts()}")

In [ ]:
# 제출 파일 생성
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': test_pred.astype(bool)
})

# CSV 파일로 저장
submission.to_csv('submission.csv', index=False)

print("제출 파일 생성 완료!")
print(f"\n제출 파일 미리보기:")
submission.head(10)

---
## 요약

### 주요 인사이트
1. **CryoSleep**이 가장 중요한 예측 변수로, 냉동수면 상태의 승객이 Transported될 확률이 높음
2. **지출 관련 변수**들도 중요한 역할을 함 - 지출이 없는 승객이 Transported될 확률이 높음
3. **HomePlanet**과 **Destination**도 유의미한 영향을 미침

### 특성 공학
- Cabin 정보에서 Deck, CabinNum, Side 추출
- PassengerId에서 그룹 정보 추출
- 총 지출액, 지출 여부, 지출 항목 수 생성
- 나이 그룹, 혼자 여행 여부 생성

### 모델 성능
- Random Forest와 Gradient Boosting 모델 비교
- 교차 검증을 통한 성능 평가